In [77]:
%run load_preprocessing.ipynb

Importation de la méthode d apprentissage non supervisée

In [54]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn import cluster 

Clustering des items [regroupement]

In [55]:
kmeans = KMeans(n_clusters=19)
kmeans.fit_predict(movie_genre)

array([ 8,  1, 10, ...,  2,  6,  4])

In [56]:
utility_matrix_clustered = []
for i in range(0, n_users):
    average = np.zeros(19)
    tmp = []
    for index in range(0, 19):
        tmp.append([])
    for j in range(0, n_items):
        #avance dans l'algo si l'utilisateur a noté l'item
        if utility_matrix[i][j] != 0:
            #reunion des items du meme genre 
            tmp[kmeans.labels_[j] - 1].append(utility_matrix[i][j])
    for index in range(0, 19):
        if len(tmp[index]) != 0:
            #calcul de la moyenne
            average[index] = np.mean(tmp[index])
        else:
            average[index] = 0
    utility_matrix_clustered.append(average)

In [ ]:
#trouver la note de moyenne globale de chaque utilisateur et la stocker dans l'objet user

In [57]:
for i in range(0, n_users):
    umc = utility_matrix_clustered[i]
    user[i].avg_r = sum(rate for rate in umc if rate > 0) / sum(rate > 0 for rate in umc)

In [58]:
def pearson_correlation_similarity(x, y , ut):
    num = 0
    den1 = 0
    den2 = 0
    
    A = ut[x - 1]
    B = ut[y - 1]
  
    num = sum((a - user[x - 1].avg_r) * (b - user[y - 1].avg_r) for a, b in zip(A, B) if a > 0 and b > 0)
    den1 = sum((a - user[x - 1].avg_r) ** 2 for a in A if a > 0)
    den2 = sum((b - user[y - 1].avg_r) ** 2 for b in B if b > 0)
    den = (den1 ** 0.5) * (den2 ** 0.5)
    if den == 0:
        return 0
    else:
        return num / den

In [59]:
pearson_correlation_similarity_matrix = np.zeros((n_users, n_users))
for i in range(0, n_users):
    for j in range(0, n_users):
        if i!=j:
            pearson_correlation_similarity_matrix[i][j] = pearson_correlation_similarity(i + 1, j + 1 , utility_matrix_clustered)

In [60]:
def norm():
    normalize = np.zeros((n_users, 19))
    for i in range(0, n_users):
        for j in range(0, 19):
            if utility_matrix_clustered[i][j] != 0:
                normalize[i][j] = utility_matrix_clustered[i][j] - user[i].avg_r
            else:
                normalize[i][j] = float('Inf')
    return normalize

In [61]:
def guess(user_id, i_id, top_n):
    similarity = []
    for i in range(0, n_users):
        if i+1 != user_id:
            similarity.append(pearson_correlation_similarity_matrix[user_id-1][i])
    temp = norm()
    temp = np.delete(temp, user_id-1, 0)
    top = [x for (y,x) in sorted(zip(similarity,temp), key=lambda pair: pair[0], reverse=True)]
    s = 0
    c = 0
    for i in range(0, top_n):
        if top[i][i_id-1] != float('Inf'):
            s += top[i][i_id-1]
            c += 1
    g = user[user_id-1].avg_r if c == 0 else s/float(c) + user[user_id-1].avg_r
    if g < 1.0:
        return 1.0
    elif g > 5.0:
        return 5.0
    else:
        return g

In [62]:
utility_copy = np.copy(utility_matrix_clustered)
for i in range(0, n_users):
    for j in range(0, 19):
        if utility_copy[i][j] == 0:
            utility_copy[i][j] = guess(i+1, j+1, 150)

In [ ]:
#prediction de la note pour u.test et calcul du MSE

In [63]:
metrics_comparison(kmeans.labels_)

Mean Squared Error of the estimator is: 1.244248
Homogeneity is : 0.229843


In [ ]:
#little UI

#user

In [80]:
#age
print("How old are you?")
age = input()

print("What is your gender?")
sex = input()

#occupation
print("What is your occupation?")
occupation = input()

print("how many movies do you want to rate?")
numb_movie = int(input())
ask = random.sample(item, numb_movie)
new_user = np.zeros(19)
average_new_user = 0
for movie in ask:
	print(movie.title + ": ")
	print(movie.imdb_url)
	a = int(input())
	average_new_user = average_new_user + a
	if new_user[kmeans.labels_[movie.id - 1]] != 0:
		new_user[kmeans.labels_[movie.id - 1]] = (new_user[kmeans.labels_[movie.id - 1]] + a) / 2
	else:
		new_user[kmeans.labels_[movie.id - 1]] = a

utility_new = np.vstack((utility_copy, new_user))

new_user = User(944 , age , sex , occupation , 110018)
user.append(new_user)

pearson_correlation_matrix = np.zeros(n_users)
list_averaging_users = []
print("Your average rate for all movies is :" +str(average_new_user/numb_movie))

#user similar
for i in range(0, n_users + 1):
    if i!=943:
        pearson_correlation_matrix[i] = pearson_correlation_similarity(944, i + 1  , utility_new)
        list_averaging_users.append(pearson_correlation_matrix[i])
    
index_max = max(enumerate(list_averaging_users),key=lambda x: x[1])[0]
print("The user similar to you is user number " +str(index_max))
#print(index_max)

user_similar = [" age : "+str(user[index_max].age) , "sex :" +str(user[index_max].sex) , "occupation:"+str(user[index_max].occupation)]
print(user_similar)

How old are you?
2
What is your gender?
2
What is your occupation?
2
how many movies do you want to rate?
2
GoldenEye (1995): 
http://us.imdb.com/M/title-exact?GoldenEye%20(1995)
2
Terminator, The (1984): 
http://us.imdb.com/M/title-exact?Terminator,%20The%20(1984)
2
Your average rate for all movies is :2.0
The user similar to you is user number 608
[' age : 13', 'sex :F', 'occupation:student', 'global average rating : 0.0']


In [72]:
user_index = []
for i in user:
	user_index.append(i.id - 1)
user_index = user_index[:943]
user_index = np.array(user_index)

top = [x for (y,x) in sorted(zip(pearson_correlation_matrix, user_index), key=lambda pair: pair[0], reverse=True)]

print("How many top genre do you want?")
numb =  int(input())

top = top[:numb]

top_genre = []

for i in range(0, numb):
	maxi = 0
	maxe = 0
	for j in range(0, 19):
		if maxe < utility_matrix[top[i]][j]:
			maxe = utility_matrix[top[i]][j]
			maxi = j
	top_genre.append(maxi)

print("Movie genres you'd like:")

for i in top_genre:
	if i == 0:
		print("unknown")
	elif i == 1:
		print("action")
	elif i == 2:
		print("adventure")
	elif i == 3:
		print("animation")
	elif i == 4:
		print("childrens")
	elif i == 5:
		print("comedy")
	elif i == 6:
		print("crime")
	elif i == 7:
		print("documentary")
	elif i == 8:
		print("drama")
	elif i == 9:
		print("fantasy")
	elif i == 10:
		print("film_noir")
	elif i == 11:
		print("horror")
	elif i == 12:
		print("musical")
	elif i == 13:
		print("mystery")
	elif i == 14:
		print("romance")
	elif i == 15:
		print("science fiction")
	elif i == 16:
		print("thriller")
	elif i == 17:
		print("war")
	else:
		print("western")

How many top genre do you want?
1
Movie genres you'd like:
drama


In [79]:
suggested_movies = max_rated_user_index_max(index_max, numb)
print("These are the top rated movies by your similar user , may take a quick look..")
print(suggested_movies)

These are the top rated movies by your similar user , may take a quick look..
             movies
0  Apollo 13 (1995)


In [ ]:
list_users = []
for usr in range(0,n_users):
    list_users.append([user[usr].sex , user[usr].age , user[usr].occupation , pearson_correlation_matrix[usr]])
    
df_users = pd.DataFrame(list_users)   
df_users.columns = ['sex' , 'age' , 'occupation' , 'similarity']

# Define some CSS to control our custom labels
css = """
table
{
  border-collapse: collapse;
}
th
{
  color: #ffffff;
  background-color: #000000;
}
td
{
  background-color: #cccccc;
}
table, th, td
{
  font-family:Arial, Helvetica, sans-serif;
  border: 1px solid black;
  text-align: right;
}
"""

fig = plt.gcf()
fig.set_size_inches(12, 10)



N = n_users
df = pd.DataFrame(index=range(N))
df['sex'] = df_users['sex']
df['age'] = df_users['age'] 
df['occupation'] = df_users['occupation']
df['similarity'] = df_users['similarity']

labels = []
for i in range(N):
    label = df.ix[[i], :].T
    label.columns = ['user :' + str(i)]
    labels.append(str(label.to_html()))



y_index_max = pearson_correlation_matrix[index_max]
points = plt.plot(df.index,pearson_correlation_matrix , 'o', color='lightblue',
                 mec='k', ms=9, mew=1, alpha=.6)
#plt.plot(df.index_max , y_index_max , color = "yellow")
plt.grid(True , alpha = 0.3)
plt.xlabel('index users' , size=15)
plt.ylabel('correlation similarity measure' , size=15)
plt.title('Users by similarities', size=15)

tooltip = plugins.PointHTMLTooltip(points[0], labels,
                                   voffset=5, hoffset=5, css=css)

plugins.connect(fig, tooltip)
mpld3.display()